In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [2]:

# Cek path saat ini
print("Sebelum:", os.getcwd())

# Set path baru
os.chdir('/home/lenov/Documents/warung')

# Cek path setelah perubahan
print("Sesudah:", os.getcwd())

Sebelum: /home/lenov/Documents/warung/app
Sesudah: /home/lenov/Documents/warung


In [4]:
print("=== Memulai Simulasi Alur dengan Dua Barang Berbeda ===")

# 0. Inisialisasi Service
produk_service = ProdukService()
transaction_service = TransactionService()
detail_service = DetailTransactionService()

# 1. Membuat atau Mengambil Dua Produk yang Berbeda
print("\n--- Langkah 1: Menyiapkan Dua Produk Berbeda ---")

id_produk_1 = 2 
produk_1 = None
try:
    produk_1 = produk_service.get_produk(id_produk_1)
    if produk_1:
        print(f"Produk 1 ditemukan: {produk_1.nama_barang} (ID: {produk_1.id}), Harga: {produk_1.harga}, Stok: {produk_1.stok}")
    else: # Jika produk tidak ada, buat produk baru
        print(f"Produk dengan ID {id_produk_1} tidak ditemukan, membuat produk baru...")
        produk_1_req = ProdukCreationRequest(
            nama_barang="Es Teh Manis", harga=5000, lokasi="Minuman Dingin",
            deskripsi_suara_lokasi="Es teh manis segar pelepas dahaga", path_qris="qris/esteh.png", stok=100
        )
        produk_1 = produk_service.create_produk(produk_1_req)
        print(f"Produk 1 baru berhasil dibuat: {produk_1}")
except Exception as e:
    print(f"Gagal menyiapkan Produk 1: {e}")
    exit()

id_produk_2 = 24 
produk_2 = None
try:
    produk_2 = produk_service.get_produk(id_produk_2)
    if produk_2:
        print(f"Produk 2 ditemukan: {produk_2.nama_barang} (ID: {produk_2.id}), Harga: {produk_2.harga}, Stok: {produk_2.stok}")
    else: # Jika produk tidak ada, buat produk baru
        print(f"Produk dengan ID {id_produk_2} tidak ditemukan, membuat produk baru...")
        produk_2_req = ProdukCreationRequest(
            nama_barang="Kentang Goreng Crispy", harga=15000, lokasi="Makanan Ringan",
            deskripsi_suara_lokasi="Kentang goreng renyah dengan saus pilihan", path_qris="qris/kentang.png", stok=50
        )
        produk_2 = produk_service.create_produk(produk_2_req)
        print(f"Produk 2 baru berhasil dibuat: {produk_2}")
except Exception as e:
    print(f"Gagal menyiapkan Produk 2: {e}")
    exit()

if not produk_1 or not produk_2:
    print("Gagal menyiapkan salah satu atau kedua produk. Simulasi dihentikan.")
    exit()

# 2. Membuat Transaksi Baru dengan Status "pending"
print("\n--- Langkah 2: Membuat Transaksi Baru dengan Status 'pending' ---")
# Estimasi total harga (bisa diset ke 0 atau estimasi awal)
estimasi_total_harga = 0  # Akan diupdate nanti berdasarkan detail
new_transaction_req = TransactionCreationRequest(
    tanggal_transaksi="2024-01-02", 
    total_harga_transaksi=estimasi_total_harga, 
    status="pending",  # Status awal: pending
    metode_pembayaran="QRIS",
    catatan=f"Simulasi pesanan: 1 {produk_1.nama_barang} & 1 {produk_2.nama_barang}."
)
transaksi_dibuat = None
try:
    transaksi_dibuat = transaction_service.create_transaction(new_transaction_req)
    print(f"Transaksi baru berhasil dibuat dengan status 'pending': {transaksi_dibuat}")
    print(f"  - ID Transaksi: {transaksi_dibuat.id}")
    print(f"  - Status: {transaksi_dibuat.status}")
    print(f"  - Total Harga Awal: {transaksi_dibuat.total_harga_transaksi}")
except Exception as e:
    print(f"Gagal membuat transaksi: {e}")
    exit()

# 3. Menambahkan Detail Transaksi
print("\n--- Langkah 3: Menambahkan Detail Transaksi ---")
detail_dibuat_list = []
total_harga_aktual_dari_detail = 0

if transaksi_dibuat:
    # Detail untuk Produk 1
    try:
        qty_produk_1 = 1 
        detail_req_1 = DetailTransactionCreationRequest(
            transaction_id=transaksi_dibuat.id,
            produk_id=produk_1.id,
            qty=qty_produk_1,
            harga_per_produk=produk_1.harga,
            total_harga_produk=qty_produk_1 * produk_1.harga
        )
        detail_dibuat_1 = detail_service.create_detail_transaction(detail_req_1)
        detail_dibuat_list.append(detail_dibuat_1)
        total_harga_aktual_dari_detail += detail_dibuat_1.total_harga_produk
        print(f"Detail transaksi untuk '{produk_1.nama_barang}' berhasil ditambahkan:")
        print(f"  - Qty: {detail_dibuat_1.qty} x {detail_dibuat_1.harga_per_produk} = {detail_dibuat_1.total_harga_produk}")
    except Exception as e:
        print(f"Gagal menambahkan detail transaksi untuk '{produk_1.nama_barang}': {e}")

    # Detail untuk Produk 2
    try:
        qty_produk_2 = 1
        detail_req_2 = DetailTransactionCreationRequest(
            transaction_id=transaksi_dibuat.id,
            produk_id=produk_2.id,
            qty=qty_produk_2,
            harga_per_produk=produk_2.harga,
            total_harga_produk=qty_produk_2 * produk_2.harga
        )
        detail_dibuat_2 = detail_service.create_detail_transaction(detail_req_2)
        detail_dibuat_list.append(detail_dibuat_2)
        total_harga_aktual_dari_detail += detail_dibuat_2.total_harga_produk
        print(f"Detail transaksi untuk '{produk_2.nama_barang}' berhasil ditambahkan:")
        print(f"  - Qty: {detail_dibuat_2.qty} x {detail_dibuat_2.harga_per_produk} = {detail_dibuat_2.total_harga_produk}")
    except Exception as e:
        print(f"Gagal menambahkan detail transaksi untuk '{produk_2.nama_barang}': {e}")

    print(f"\nTotal harga dari semua detail transaksi: {total_harga_aktual_dari_detail}")

    # 4. Update Status Transaksi menjadi "success" dan Total Harga berdasarkan Detail
    print("\n--- Langkah 4: Update Status Transaksi menjadi 'success' dan Total Harga ---")
    if detail_dibuat_list:  # Hanya update jika ada detail yang berhasil ditambahkan
        try:
            print(f"Memperbarui transaksi (ID: {transaksi_dibuat.id}):")
            print(f"  - Status: 'pending' -> 'success'")
            print(f"  - Total Harga: {transaksi_dibuat.total_harga_transaksi} -> {total_harga_aktual_dari_detail}")
            
            # Update transaksi dengan total harga dari detail dan status success
            update_req = TransactionCreationRequest(
                tanggal_transaksi=transaksi_dibuat.tanggal_transaksi,
                total_harga_transaksi=total_harga_aktual_dari_detail,  # Update dengan total dari detail
                status="success",  # Update status menjadi success
                metode_pembayaran=transaksi_dibuat.metode_pembayaran,
                catatan=transaksi_dibuat.catatan + " | Status diperbarui ke success, total dihitung dari detail."
            )
            transaksi_terupdate = transaction_service.update_transaction(transaksi_dibuat.id, update_req)
            transaksi_dibuat = transaksi_terupdate
            print(f"Transaksi berhasil diperbarui:")
            print(f"  - ID: {transaksi_terupdate.id}")
            print(f"  - Status: {transaksi_terupdate.status}")
            print(f"  - Total Harga: {transaksi_terupdate.total_harga_transaksi}")
        except Exception as e:
            print(f"Gagal memperbarui transaksi: {e}")
    else:
        print("Tidak ada detail transaksi yang berhasil ditambahkan, transaksi tetap berstatus 'pending'.")
else:
    print("Tidak dapat melanjutkan: Transaksi belum berhasil dibuat.")

# 5. Mengambil dan Menampilkan Transaksi Final dengan Detailnya
print("\n--- Langkah 5: Menampilkan Transaksi Final ---")
if transaksi_dibuat:
    try:
        transaksi_final = transaction_service.get_transaction(transaksi_dibuat.id)
        if transaksi_final:
            print(f"\n{'='*60}")
            print(f"TRANSAKSI FINAL (ID: {transaksi_final.id})")
            print(f"{'='*60}")
            print(f"Tanggal          : {transaksi_final.tanggal_transaksi}")
            print(f"Status           : {transaksi_final.status}")
            print(f"Metode Pembayaran: {transaksi_final.metode_pembayaran}")
            print(f"Total Harga      : Rp {transaksi_final.total_harga_transaksi:,}")
            print(f"Catatan          : {transaksi_final.catatan}")

            # Menampilkan detail transaksi
            print(f"\nDETAIL PESANAN:")
            print(f"{'-'*60}")
            if detail_dibuat_list:
                calculated_total_from_details = 0
                for i, detail_obj in enumerate(detail_dibuat_list, 1):
                    # Dapatkan info produk
                    produk_info = produk_1 if detail_obj.produk_id == produk_1.id else produk_2
                    nama_produk = produk_info.nama_barang if produk_info else f"Produk ID {detail_obj.produk_id}"
                    
                    print(f"{i}. {nama_produk}")
                    print(f"   Jumlah    : {detail_obj.qty}")
                    print(f"   Harga/item: Rp {detail_obj.harga_per_produk:,}")
                    print(f"   Subtotal  : Rp {detail_obj.total_harga_produk:,}")
                    calculated_total_from_details += detail_obj.total_harga_produk
                    print()
                
                print(f"{'-'*60}")
                print(f"TOTAL DARI DETAIL: Rp {calculated_total_from_details:,}")
                print(f"TOTAL DI DATABASE: Rp {transaksi_final.total_harga_transaksi:,}")
                
                if calculated_total_from_details == transaksi_final.total_harga_transaksi:
                    print("✅ Total transaksi cocok dengan jumlah detail!")
                else:
                    print("❌ PERHATIAN: Total tidak cocok!")
            else:
                print("Tidak ada detail transaksi.")
        else:
            print(f"Gagal mengambil transaksi final (ID: {transaksi_dibuat.id}).")
    except Exception as e:
        print(f"Gagal menampilkan transaksi final: {e}")
else:
    print("Transaksi tidak berhasil dibuat.")

print(f"\n{'='*60}")
print("=== SIMULASI SELESAI ===")
print(f"{'='*60}")
print("RINGKASAN ALUR:")
print("1. ✅ Buat transaksi dengan status 'pending' dan total harga 0")
print("2. ✅ Tambahkan detail transaksi untuk setiap produk")
print("3. ✅ Hitung total harga dari semua detail transaksi")
print("4. ✅ Update status transaksi menjadi 'success'")
print("5. ✅ Update total harga transaksi dengan total dari detail")
print("6. ✅ Tampilkan hasil akhir")

=== Memulai Simulasi Alur dengan Dua Barang Berbeda ===


NameError: name 'ProdukService' is not defined

In [ ]:
print(produk_service.get_all_produk()[-1].stok)
print(produk_service.get_produk(2))
print(produk_service.get_produk(24))
for i in detail_service.get_detail_transactions_by_transaction_id(34):
    print(i)
print(transaction_service.get_all_transactions()[-1])

NameError: name 'produk_service' is not defined

In [23]:
from agents.mcp import MCPServerStdio
from agents import Agent, Runner, trace
from IPython.display import Markdown, display

params_produk = {"command": "uv", "args": ["run", "app/mcp_sample/produk_server.py"]}
async with MCPServerStdio(params=params_produk) as server:
    mcp_produk = await server.list_tools()

params_transaction = {"command": "uv", "args": ["run", "app/mcp_sample/transaction_server.py"]}
async with MCPServerStdio(params=params_transaction) as server:
    mcp_transaction = await server.list_tools()

print("=================")
print("MCP PRODUK")
print("=================")
for i in mcp_produk:
    print(i.name, " --> ", i.description)
print("\n=================")
print("MCP TRANSACTION")
print("=================")
for i in mcp_transaction:
    print(i.name, " --> ", i.description)

MCP PRODUK
list_all_produk  -->  List all available products.

MCP TRANSACTION
create_transaction  -->  Create a new transaction with its detail transactions.

    Args:
        tanggal_transaksi (str): The date of the transaction in YYYY-MM-DD format.
        total_harga_transaksi (int): The total amount of the transaction.
        status (str): The current status of the transaction (e.g., "pending", "success", "failed").
        metode_pembayaran (str): The payment method used for the transaction.
        detail_transactions (List[DetailTransactionCreationRequest]): A list of dictionaries, where each dictionary represents a detail transaction.
            Example: '''
            [
                {"produk_id": 1, "qty": 2, "harga_per_produk": 15000, "total_harga_produk": 30000},
                {"produk_id": 2, "qty": 1, "harga_per_produk": 25000, "total_harga_produk": 25000}
            ]
            '''
        catatan (Optional[str]): Optional notes or remarks for the transaction

In [8]:
with open(f"{os.getcwd()}/app/instruction.md", 'r', encoding='utf-8') as file:
        instructions = file.read()
request = f"halo saya mau beli Lampu led 10w sebanyak 2"
model = "phi4"
instructions

'Anda adalah AI Asisten untuk Warung Mandiri yang cerdas dan memiliki ingatan.\n\n### Prinsip Utama\n\n1.  **Anda Memiliki Memori**: Anda menerima riwayat percakapan (`messages`) dalam state. Gunakan konteks ini untuk memahami permintaan pengguna yang singkat (misalnya, jika pengguna bertanya "harga susu", lalu mengetik "2", Anda harus mengerti "2" adalah kuantitas susu yang ingin dibeli).\n2.  **Manajemen State**: Informasi penting seperti `transaction_id` dan `cart_items` disimpan di dalam state. Tugas Anda adalah memanggil *tools* yang tepat untuk memperbarui state ini.\n3.  **Pandu Pengguna**: Beri tahu pengguna apa yang terjadi pada setiap langkah dan apa yang bisa mereka lakukan selanjutnya.\n\n### Alur Pemrosesan Transaksi (Stateful)\n\n**Langkah 1: Pencarian dan Penambahan Produk ke Keranjang**\n\n* Ketika pengguna menanyakan produk (misal: "Ada Indomie Goreng?"), gunakan tool `get_produk_by_name`.\n* Jika ditemukan, berikan informasi harga dan stok. Tanyakan berapa banyak yang

In [19]:
!ollama list

NAME                           ID              SIZE      MODIFIED     
qwen2.5:latest                 845dbda0ea48    4.7 GB    26 hours ago    
llama3.1:latest                46e0c10c039e    4.9 GB    27 hours ago    
llama3.1:8b                    46e0c10c039e    4.9 GB    27 hours ago    
llama3-groq-tool-use:latest    36211dad2b15    4.7 GB    5 days ago      


In [22]:
from ollama import Client
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI

url_kantor = "http://172.20.3.3:11434/v1"
url_local = "http://localhost:11434/v1"

model = OpenAIChatCompletionsModel( 
    model="llama3.1:latest",
    openai_client=AsyncOpenAI(base_url=url_local)
)
async with MCPServerStdio(params=params_produk) as mcp_produk:
    async with MCPServerStdio(params=params_transaction) as mcp_transaction:
        agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_produk, mcp_transaction])
        with trace("conversation"):
            result = await Runner.run(agent, request)
        display(Markdown(result.final_output))

Errored MCP tool result: meta=None content=[] isError=False


Maaf, kami tidak menemukan informasi tentang produk "Lampu LED 10W" di database kami. Apakah Anda ingin mencari lagi?

Apabila Anda mengetahui harga dan stok Lampu LED 10W terbaru, silakan kirimkan informasinya ke saya, dapatkah saya membantu menambahkannya ke keranjang belanja Anda?

In [11]:
import gradio as gr
import os

async def warung_ai_chat_with_history(message: str, chat_history: list):
    """
    Version that includes chat history context
    """
    try:
        async with MCPServerStdio(params=params_produk) as mcp_produk:
            async with MCPServerStdio(params=params_transaction) as mcp_transaction:
                
                # Build conversation context from history
                conversation_context = ""
                if chat_history:
                    conversation_context = "\n\nKonteks Percakapan Sebelumnya:\n"
                    for i, (user_msg, ai_response) in enumerate(chat_history):
                        conversation_context += f"User: {user_msg}\nAI: {ai_response}\n\n"
                    conversation_context += "---Percakapan Berlanjut---\n"
                
                # Add context to current message
                full_message = conversation_context + f"User: {message}"
                
                agent = Agent(
                    name="WarungAI_Agent_Gradio", 
                    instructions=instructions, 
                    model=model, 
                    mcp_servers=[mcp_produk, mcp_transaction]
                )
                
                # trace("percakapan_gradio")
                result = await Runner.run(agent, full_message, max_turns=50) 
                
                return result.final_output
                
    except Exception as e:
        print(f"Error dalam warung_ai_chat: {e}")
        return f"Maaf, terjadi kesalahan internal pada server: {str(e)}"


if __name__ == "__main__":
    # Use the first solution
    iface = gr.ChatInterface(
        fn=warung_ai_chat_with_history,  # <-- Use the fixed function
        title="Warung AI Assistant (MCP Demo)",
        description=(
            "Selamat datang di Warung AI! Ketik pesan Anda untuk berinteraksi.\n"
            "AI akan mengingat konteks percakapan sebelumnya.\n"
            "Contoh alur:\n"
            "1. 'Saya mau beli lampu' → AI akan cari produk\n"
            "2. '2' → AI akan proses sebagai kuantitas untuk produk sebelumnya\n"
        ),
        chatbot=gr.Chatbot(height=550, label="Percakapan", show_label=True, show_copy_button=True),
        textbox=gr.Textbox(placeholder="Ketik pesan ke Warung AI...", container=False, scale=7, label="Input Pengguna"),
        theme="soft",
        examples=[
            ["Saya mau cek produk Lampu"],
            ["Ada tambahan barang lain?"],
            ["Lanjut pembayaran"],
        ],
        submit_btn=True,
        autofocus=True,
    )
    
    print("Meluncurkan Gradio Interface dengan Context History...")
    iface.launch(show_error=True)

/tmp/ipykernel_69633/2161259028.py:61: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=550, label="Percakapan", show_label=True, show_copy_button=True),
/home/lenov/Documents/warung/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Meluncurkan Gradio Interface dengan Context History...
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


chat_history: []
message: Saya mau cek produk Lampu
full_message with context: User: Saya mau cek produk Lampu...
result.final_output: Maaf, produk **Lampu** tidak ditemukan. Apakah ada produk lain yang ingin Anda cek?
chat_history: []
message: Saya mau cek produk Lampu
full_message with context: User: Saya mau cek produk Lampu...
result.final_output: Maaf, produk "Lampu" tidak ditemukan. Stok untuk barang tersebut saat ini tidak tersedia. Apakah ada barang lain yang ingin Anda cari?
chat_history: [['Saya mau cek produk Lampu', 'Maaf, produk "Lampu" tidak ditemukan. Stok untuk barang tersebut saat ini tidak tersedia. Apakah ada barang lain yang ingin Anda cari?']]
message: kopi aja
full_message with context: 

Konteks Percakapan Sebelumnya:
User: Saya mau cek produk Lampu
AI: Maaf, produk "Lampu" tidak ditemukan. Stok untuk barang tersebut saat ini tidak tersedia. Apakah ada barang lain yang ingin Anda ca...
result.final_output: Produk "Kopi Sachet ABC Susu" ditemukan. Harganya Rp 1.50